In [ ]:
import sys
sys.path.append('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/')

from util.util_eval import * 

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report
from sklearn.preprocessing import label_binarize

import pandas as pd 
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

wd = Path('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/validation/data/qprj_pred_all_PDO_wsi/')
wd_fig = wd.joinpath('roc_pdf')
type_path = Path('/Users/JIC115/Desktop/CBS_prj/Rose_PDO_model/util/type_info_PDO_3chl.json')

## 1. Generate cell level prediction table

In [ ]:
### get cell info for each gs2 with PDO info from gs1
gsdir1 = wd.joinpath('rose_gson_modified2')
gsdir2 = wd.joinpath('PDO_wsi_all_chk9ep32_5x_gson')
outdir = wd.joinpath('pred_sunny_tables/Cell-level_csv')
celltypes = ['background','normal','tumor']

gs2table_PDO(outdir,gsdir1,gsdir2,celltypes)

### 2.1 Split data

In [ ]:
### extract train slides
train_wsi = ['HN23-10730-p4*',  'HN23-10752-p1*', 'HN24-10810-p1*', 'HN24-10909-P2*',
             'HN23-10752-p2*' , 'HN24-10791-p1*', 'HN24-10831-p3*']

indir  = wd.joinpath('pred_sunny_tables/Cell-level_csv')
outdir = wd.joinpath('pred_sunny_tables/Cell-level_train')
for regex in train_wsi:
    copy_move_files(outdir,indir,copy=False,regex=regex,rename = None,ds_pct = 0)

indir.rename(indir.parent.joinpath('Cell-level_test'))  # rename original folder

In [ ]:
### extract same pts
train_pt = ['HN23-10730*','HN23-10752*', 'HN24-10810*', 'HN24-10909*',
             'HN23-10752*' , 'HN24-10791*', 'HN24-10831*']

indir = wd.joinpath('pred_sunny_tables/Cell-level_test')
outdir = wd.joinpath('pred_sunny_tables/Cell-level_same_pts')
for regex in train_pt:
    copy_move_files(outdir,indir,copy=False,regex=regex,rename = None,ds_pct = 0)

indir.rename(indir.parent.joinpath('Cell-level_diff_pts'))

### 2.2 Gen table for training slides

In [ ]:
indir = wd.joinpath('pred_sunny_tables/Cell-level_train')
print(len(sorted(indir.iterdir())))
df = pd.DataFrame()

for f in sorted(indir.iterdir()):
    df = pd.concat([df, pd.read_csv(str(f))]) 
    
df.rename(columns = {'0':'prob_background','1':'prob_normal','2':'prob_tumor'},inplace=True)
df.to_csv(wd.joinpath('pred_sunny_tables/Cell-level_train.csv'))

In [ ]:
tbl_fn = wd.joinpath('pred_sunny_tables/Cell-level_train.csv')
df = pd.read_csv(str(tbl_fn))

# remove background
df = df.query("pred_type != 'background' and true_type != 'background'")
print(df.pred_type.unique())
print(df.true_type.unique())

####### make plots
ytrue = df.true_type 
ypred = df.pred_type

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)

#-------- ROC plot
yprob = df.prob_tumor
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd_fig.joinpath('Cell-level_train_roc.pdf'))

#-------- report table
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report


### 2.3 Gen table for same patients

In [ ]:
### gen table for test ###
indir = wd.joinpath('pred_sunny_tables/Cell-level_same_pts')
print(len(sorted(indir.iterdir())))
df = pd.DataFrame()

for f in sorted(indir.iterdir()):
    df = pd.concat([df, pd.read_csv(str(f))]) 
    
df.rename(columns = {'0':'prob_background','1':'prob_normal','2':'prob_tumor'},inplace=True)
df.to_csv(wd.joinpath('pred_sunny_tables/Cell-level_same_pts.csv'))

In [ ]:
tbl_fn = wd.joinpath('pred_sunny_tables/Cell-level_same_pts.csv')
df = pd.read_csv(str(tbl_fn))

# remove background
df = df.query("pred_type != 'background' and true_type != 'background'")
print(df.pred_type.unique())
print(df.true_type.unique())

####### make plots
ytrue = df.true_type 
ypred = df.pred_type

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)

#-------- ROC plot
yprob = df.prob_tumor
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd_fig.joinpath('Cell-level_same_pts_roc.pdf'))

#-------- report table
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report

### 2.4 Gen table for diff patients

In [ ]:
indir = wd.joinpath('pred_sunny_tables/Cell-level_diff_pts')
print(len(sorted(indir.iterdir())))
df = pd.DataFrame()

for f in sorted(indir.iterdir()):
    df = pd.concat([df, pd.read_csv(str(f))]) 
    
df.rename(columns = {'0':'prob_background','1':'prob_normal','2':'prob_tumor'},inplace=True)
df.to_csv(wd.joinpath('pred_sunny_tables/Cell-level_diff_pts.csv'))

In [ ]:
tbl_fn = wd.joinpath('pred_sunny_tables/Cell-level_diff_pts.csv')
df = pd.read_csv(str(tbl_fn))

# remove background
df = df.query("pred_type != 'background' and true_type != 'background'")
print(df.pred_type.unique())
print(df.true_type.unique())

####### make plots
ytrue = df.true_type 
ypred = df.pred_type

#-------- confusion plot
conf_mat = confusion_matrix(ytrue, ypred) # sorted by alphabeta order of labels !!!
label = sorted(['normal','tumor']) 
cmat_disp = ConfusionMatrixDisplay(conf_mat,display_labels=label)

# customize :
fig, ax = plt.subplots(figsize=(6.5,6.6))    # set plot figsize
cmat_disp.plot(ax=ax, colorbar=False)    # Deactivate default colorbar,size does not match with plot

# Adding custom colorbar
cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
plt.colorbar(cmat_disp.im_,  cax=cax)

#-------- ROC plot
yprob = df.prob_tumor
RocCurveDisplay.from_predictions(ytrue,yprob,pos_label='tumor')
plt.savefig(wd_fig.joinpath('Cell-level_diff_pts_roc.pdf'))

#-------- report table
report = pd.DataFrame(classification_report(ytrue, ypred, output_dict = True))
report.rename(index = {'recall':'sensitivity'},inplace=True)
report